In [1]:
%matplotlib inline
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

import sys, os
# sys.path.append('../largescale_smc/')
# sys.path.append('../third_party')

In [3]:
import numpy as np   
import pandas as pd
import pdb
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from tqdm import tqdm
import sys

from time import time
from largescale_smc.largescale_missingness import *
from largescale_smc.largescale_utils import *
from largescale_smc.largescale_method import *

In [4]:
from surprise import SVD
from surprise import accuracy

In [5]:
%%memit
start_time = time()
#file_path = os.path.expanduser("../data/ml-100k/u.data")
file_path = os.path.expanduser("../data/ml-10m/ratings_no_timestamp.dat")
reader = DataReader(line_format="user item rating", sep="::")
splitter = DataSplitter(file_path, reader)
train_samples, calib_samples, missing_idxs = splitter.sample_train_calib(k=5)
print(f"Elapsed time: {time()-start_time} seconds")

10000054it [00:14, 698209.62it/s]

Loaded 10000054 ratings from 69878 users and 10677 items.



100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:03<00:00, 590.29it/s]


Elapsed time: 33.60444211959839 seconds
peak memory: 2562.42 MiB, increment: 2401.51 MiB


In [6]:
%%memit
random_state=0
start_time = time()
trainset = construct_trainset(train_samples, reader.rating_scale)
algo = SVD(n_factors=10, n_epochs=10, biased=False, verbose=True)
algo.fit(trainset)
print(f"Fitting time: {time()-start_time} seconds")

# start_time = time()
# # Then compute RMSE
# predictions = algo.test(raw_testset)
# accuracy.rmse(predictions)
# print(f"Testing time: {time()-start_time} seconds")

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Fitting time: 74.14759016036987 seconds
peak memory: 4036.02 MiB, increment: 1559.23 MiB


In [7]:
%%memit
start_time = time()
w_obs=LogisticMFProbs(random_state=random_state)
w_obs.fit(train_samples+calib_samples)
print(f"Elapsed time: {time()-start_time} seconds")

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:22<00:00,  1.34it/s]


Elapsed time: 61.350459575653076 seconds
peak memory: 4666.21 MiB, increment: 779.15 MiB


In [8]:
%%memit
start_time = time()
k=5
Mhat = create_batch_rating_predictor(algo)
ci_method = SimulCI_ls(splitter.config, calib_samples, k, Mhat, w_obs)
print(f"Elapsed time: {time()-start_time} seconds")

Elapsed time: 0.0416417121887207 seconds
peak memory: 3906.58 MiB, increment: 1.09 MiB


In [9]:
calib_idxs = [(u, i) for u,i,r in calib_samples]
calib_ratings = [r for u,i,r in calib_samples]

In [13]:
%%memit
start_time = time()
alpha = 0.1
allow_inf= False
df = ci_method.get_CI(calib_idxs[:k], alpha, allow_inf=allow_inf)
print(f"Elapsed time: {time()-start_time} seconds")

Computing conformal prediction intervals for 1 test queries...
Start computing universal quantities invariant to the test query...


Universal quantities: 100%|██████████████████████████████████████████████████████| 10677/10677 [23:08<00:00,  7.69it/s]


Done!


CI:   0%|                                                                                        | 0/1 [00:00<?, ?it/s]


TypeError: 'function' object is not subscriptable

In [ ]:
plt.hist(w_obs.mean_user_factor)

In [ ]:
plt.hist(w_obs.user_factors[1])

In [ ]:
w_obs.user_factors[1]

In [ ]:
w_obs.mean_user_factor

In [ ]:
alph = ['a', 'b', 'c']

In [ ]:
num = [1,2,3]

In [ ]:
tt=list(zip(alph, num))

In [ ]:
tt[0][1]

In [ ]:
np.abs([1,2,4]-[0,1,1])

In [ ]:
import sys

In [ ]:
sys.float_info.max

In [ ]:
ss=set([1,23,4,5,1])

In [ ]:
np.prod(np.array([1,23,4]))

In [ ]:
test= 1

In [ ]:
test /= 1+1

In [ ]:
test

In [ ]:
np.array([1,0,0]) | np.array([1,0,1])